In [17]:
import os
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [13]:
dataset = pd.read_csv("../../data/BG dataset.csv")

In [14]:
dataset

,file_name,time_stamp,glucose,time_lag,time_sec
0,S001,2019-04-16 14:14:03,111,NaN,NaN
1,S001,2019-04-16 14:18:58,112,295s (~4.92 minutes),295.0
2,S001,2019-04-16 14:23:58,112,300s (~5 minutes),300.0
3,S001,2019-04-16 14:28:58,112,300s (~5 minutes),300.0
4,S001,2019-04-16 14:33:58,112,300s (~5 minutes),300.0
...,...,...,...,...,...
156552,S142,2023-03-14 19:18:48,352,301s (~5.02 minutes),301.0
156553,S142,2023-03-14 19:23:47,357,299s (~4.98 minutes),299.0
156554,S142,2023-03-14 19:28:48,356,301s (~5.02 minutes),301.0
156555,S142,2023-03-14 19:33:48,354,300s (~5 minutes),300.0


In [15]:
patients = dataset.file_name.unique()
np.random.shuffle(patients)
patients_fold = np.array_split(patients, 10)

In [18]:
train_fold = []
test_fold = []
for i in range(10):
    fold = copy.deepcopy(patients_fold)
    test_fold.append(fold[i].tolist())
    del fold[i]
    train_fold.append(np.concatenate(fold).tolist())

In [25]:
np.savez("../../data/BG_Fold_Info.npz", train=train_fold, test=test_fold)

In [12]:
from sklearn.model_selection import TimeSeriesSplit
from scipy.stats import skew, kurtosis

#import tsfresh
#from tsfresh.feature_extraction import ComprehensiveFCParameters
#from tsfresh.utilities.dataframe_functions import impute

In [13]:
def gen_data(glucose_data, train_size, gap):
    
    tscv = TimeSeriesSplit(n_splits = len(glucose_data)-train_size-gap,
                           max_train_size = train_size,
                           test_size = 1,
                           gap = gap)
    train_list=[]
    test_list=[]
    
    for train_index, test_index in tscv.split(glucose_data):
        train, test = glucose_data[train_index], glucose_data[test_index]
        train_list.append(train.tolist())
        test_list.append(test.tolist()[0])
    
    return(train_list, test_list)

In [14]:
directory = "../../data/BG Ex data/"

if not os.path.exists(directory):
    os.makedirs(directory)

for fn in dataset.file_name.unique():
    glucose_data = dataset[dataset["file_name"]==fn].reset_index(drop=True)
    
    gen12_x, gen12_y = gen_data(glucose_data.glucose, 12, 5)
    gen24_x, gen24_y = gen_data(glucose_data.glucose, 24, 5)
    gen36_x, gen36_y = gen_data(glucose_data.glucose, 36, 5)
    
    gen12 = pd.DataFrame()
    for i, ts in enumerate(gen12_x):
        data = [[x,i] for x in ts]
        gen12 = gen12.append(data, ignore_index=True)
    gen12.columns=["value", "id"]
    
    gen24 = pd.DataFrame()
    for i, ts in enumerate(gen24_x):
        data = [[x,i] for x in ts]
        gen24 = gen24.append(data, ignore_index=True)
    gen24.columns=["value", "id"]
    
    gen36 = pd.DataFrame()
    for i, ts in enumerate(gen36_x):
        data = [[x,i] for x in ts]
        gen36 = gen36.append(data, ignore_index=True)
    gen36.columns=["value", "id"]
    
    #extraction_settings = ComprehensiveFCParameters()
    
    #tf=tsfresh.extract_features(gen12, 
    #                            column_id="id",
    #                            default_fc_parameters=extraction_settings,
    #                            impute_function=impute)
    #tf12 = tf.T.drop_duplicates().T
    
    #tf=tsfresh.extract_features(gen24, 
    #                            column_id="id",
    #                            default_fc_parameters=extraction_settings,
    #                            impute_function=impute)
    #tf24 = tf.T.drop_duplicates().T
    
    #tf=tsfresh.extract_features(gen36, 
    #                            column_id="id",
    #                            default_fc_parameters=extraction_settings,
    #                            impute_function=impute)
    #tf36 = tf.T.drop_duplicates().T
    
    np.savez_compressed("../../data/BG Ex data/"+fn+".npz",
                        gen12 = gen12_x[24:],
                        #tf12 = tf12[24:],
                        y12 = gen12_y[24:],
                        #tf12_name = tf12.columns.tolist(),
                        gen24 = gen24_x[12:],
                        #tf24 = tf24[12:],
                        y24 = gen24_y[12:],
                        #tf24_name = tf24.columns.tolist(),
                        gen36 = gen36_x,
                        #tf36 = tf36,
                        y36 = gen36_y#,
                        #tf36_name = tf36.columns.tolist()
                       )

C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen12 = gen12.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen24 = gen24.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen36 = gen36.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen12 = gen12.append(data, ignore_index=True)
C:\Users\use

C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen24 = gen24.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen36 = gen36.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen12 = gen12.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen24 = gen24.append(data, ignore_index=True)
C:\Users\use

C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen12 = gen12.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen24 = gen24.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen36 = gen36.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen12 = gen12.append(data, ignore_index=True)
C:\Users\use

C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen24 = gen24.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen36 = gen36.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen12 = gen12.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen24 = gen24.append(data, ignore_index=True)
C:\Users\use

C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen36 = gen36.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen12 = gen12.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen24 = gen24.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen36 = gen36.append(data, ignore_index=True)
C:\Users\use

...

C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen36 = gen36.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen12 = gen12.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen24 = gen24.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen36 = gen36.append(data, ignore_index=True)
C:\Users\use

C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen12 = gen12.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen24 = gen24.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen36 = gen36.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen12 = gen12.append(data, ignore_index=True)
C:\Users\use

C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen36 = gen36.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen12 = gen12.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen24 = gen24.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen36 = gen36.append(data, ignore_index=True)
C:\Users\use

C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen12 = gen12.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen24 = gen24.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen36 = gen36.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen12 = gen12.append(data, ignore_index=True)
C:\Users\use

C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen24 = gen24.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen36 = gen36.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen12 = gen12.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30932\766280793.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen24 = gen24.append(data, ignore_index=True)
C:\Users\use